In [1]:
!pip install sounddevice soundfile
import sounddevice as sd
import soundfile as sf
import numpy as np 

In [5]:
import sounddevice as sd

# Liệt kê toàn bộ thiết bị âm thanh
devices = sd.query_devices()

# In danh sách thiết bị
print("Danh sách thiết bị âm thanh có sẵn:\n")
for i, device in enumerate(devices):
    print(f"{i}: {device}")


Danh sách thiết bị âm thanh có sẵn:

0: {'name': 'Microsoft Sound Mapper - Input', 'index': 0, 'hostapi': 0, 'max_input_channels': 2, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
1: {'name': 'Microphone (DGM20 USB Microphon', 'index': 1, 'hostapi': 0, 'max_input_channels': 2, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
2: {'name': 'Microphone (DroidCam Virtual Au', 'index': 2, 'hostapi': 0, 'max_input_channels': 1, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
3: {'name': 'Microphone (e2eSoft iVCam)', 'index': 3, 'hostapi': 0,

In [48]:
import sounddevice as sd
import soundfile as sf
import os

# Cấu hình ghi âm
device_index = 1  # Chọn Microphone Array (AMD Audio Dev)
duration = 6  # Ghi âm trong 5 giây
fs = 16000  # ⚠️ Thay đổi: Tần số lấy mẫu 16kHz, chuẩn cho voice identification

# Chọn user để ghi âm
user_name = "user_C"  # Thay đổi thành "user_B" nếu ghi âm cho user B
save_dir = f"data/{user_name}"

# Tạo thư mục nếu chưa có
os.makedirs(save_dir, exist_ok=True)

# Tìm số thứ tự file tiếp theo
existing_files = [f for f in os.listdir(save_dir) if f.endswith(".wav")]
file_count = len(existing_files) + 1
output_filename = f"{save_dir}/{user_name}_{file_count}.wav"

# Bắt đầu ghi âm
print(f"Bắt đầu ghi âm cho {user_name}, lưu vào {output_filename}...")
recording = sd.rec(
    int(duration * fs), samplerate=fs, channels=1, dtype="int16", device=device_index
)
sd.wait()  # Chờ ghi âm hoàn tất
print("Ghi âm xong!")

# Lưu file WAV
sf.write(output_filename, recording, fs)
print(f"Đã lưu file: {output_filename}")

Bắt đầu ghi âm cho user_C, lưu vào data/user_C/user_C_40.wav...
Ghi âm xong!
Đã lưu file: data/user_C/user_C_40.wav


In [50]:
import os

print("is file:", os.path.isfile("./model/improved_speaker_model_v1.h5"))
print("is dir:", os.path.isdir("./model/improved_speaker_model_v1.h5"))


is file: True
is dir: False


In [48]:
import sounddevice as sd
import numpy as np
import librosa
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle
model = tf.keras.models.load_model("./model/improved_speaker_model_v1.h5")


ValueError: Unrecognized keyword arguments: ['batch_shape']

In [51]:
import sounddevice as sd
import numpy as np
import librosa
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle

# --- Parameters (update if needed) ---
SAMPLE_RATE = 44100
DURATION = 5  # seconds
N_MFCC = 40
MAX_LENGTH = 200  # number of time steps (must match training)

# --- Load model and label encoder ---
model = tf.keras.models.load_model("../ai_model/model/improved_speaker_model_v1.h5", compile=False)


# If you saved label encoder using pickle
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# --- Record audio ---
print("🎤 Recording for {} seconds...".format(DURATION))
audio = sd.rec(int(SAMPLE_RATE * DURATION), samplerate=SAMPLE_RATE, channels=1)
sd.wait()
print("✅ Done recording!")

# --- Convert audio to MFCC ---
audio = np.squeeze(audio)  # shape: (samples,)
mfcc = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=N_MFCC)

# --- Pad or truncate to MAX_LENGTH ---
if mfcc.shape[1] < MAX_LENGTH:
    pad_width = MAX_LENGTH - mfcc.shape[1]
    mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
else:
    mfcc = mfcc[:, :MAX_LENGTH]

mfcc = mfcc.T[np.newaxis, ...]  # shape: (1, MAX_LENGTH, N_MFCC)

# --- Predict ---
pred_probs = model.predict(mfcc)
pred_index = np.argmax(pred_probs)
pred_label = label_encoder.inverse_transform([pred_index])[0]

print(f"🔊 Predicted Speaker: {pred_label}")


ValueError: Unrecognized keyword arguments: ['batch_shape']